In [ ]:
from IPython.display import display, Math
import sympy as sp
from sympy.parsing.sympy_parser import parse_expr
import warnings
warnings.filterwarnings('ignore')

: 

# Punto 2

In [ ]:
x = parse_expr('x') # añade la varible
f = parse_expr('x*ln(x)')  # crea la funcion reespecto a la variable
x0 = 0
n = 6

p = sp.series(expr=f, x=x, x0=x0, n=n)
display(Math(sp.latex(p)))

: 

# punto 3

In [ ]:
x = parse_expr('x') # añade la varible
f = parse_expr('sin(x)')  # crea la funcion reespecto a la variable
x0 = 0
n = 6

: 

In [ ]:
p = sp.series(expr=f, x=x, x0=x0, n=n)
display(Math(sp.latex(p)))

: 

In [ ]:
max_n = 10
p = [sp.series(expr=f, x=x, x0=x0, n=n).removeO() for n in range(4,max_n)]
p_latex = [sp.latex(pol) for pol in p]
distLatex = [display(Math(q)) for q in p_latex]
n_pol = len(p)

: 

In [ ]:
import matplotlib.pyplot as plt
import sympy.plotting as pls
import seaborn as sns
from numpy import pi

: 

In [ ]:
sns.set_style('whitegrid')
cmap = sns.color_palette('Blues', n_pol)
sns.palplot(cmap)

: 

In [ ]:
fig = pls.plot(f,xlim=[-5*pi,5*pi],ylim=[-5*pi,5*pi], line_color='r', show=False)
labs = ['$f(x)$']
for n in range(n_pol):
  c = cmap[n]
  labs.append(f'$p_{n+1}(x)$')
  fig.extend(pls.plot(p[n],xlim=[-5*pi,5*pi],ylim=[-5*pi,5*pi], line_color=c, show=False))
for n in range(n_pol+1): fig[n].label = labs[n]
fig.legend = True
fig.show()

: 